In [1]:
import numpy as np
import torch

# 加载Fashion-MNIST数据集
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
# 定义softmax操作
def softmax(X):
    exps = torch.exp(X)
    return exps / torch.sum(exps, dim=1, keepdim=True)

In [3]:
# 定义模型
def model(X):
    logits = torch.matmul(X, w) + b
    return softmax(logits)

In [4]:
# 定义交叉熵损失函数
def cross_entropy(y_pred, y_true):
    y_true = torch.LongTensor(y_true)
    log_probs = -torch.log(y_pred.gather(1, y_true.view(-1, 1)))
    return torch.mean(log_probs)


In [5]:
# 定义准确率计算函数
def accuracy(y_pred, y_true):
    y_pred = torch.argmax(y_pred, dim=1)
    y_true = torch.LongTensor(y_true)
    return torch.mean((y_pred == y_true).float())

In [6]:
# 定义训练函数
def train(epochs, learning_rate):
    global w, b
    for epoch in range(epochs):
        total_loss = 0
        total_accuracy = 0
        for batch_images, batch_labels in train_dataloader:
            batch_images = batch_images.reshape(-1, num_features)
            
            # 前向传播
            y_pred = model(batch_images)
            
            # 计算损失和准确率
            loss = cross_entropy(y_pred, batch_labels)
            acc = accuracy(y_pred, batch_labels)
            
            # 反向传播和参数更新
            loss.backward()
            with torch.no_grad():
                w -= learning_rate * w.grad
                b -= learning_rate * b.grad
                w.grad.zero_()
                b.grad.zero_()
            
            total_loss += loss.item()
            total_accuracy += acc.item()
        
        avg_loss = total_loss / len(train_dataloader)
        avg_accuracy = total_accuracy / len(train_dataloader)
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

In [7]:

# 定义测试函数
def test():
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for batch_images, batch_labels in test_dataloader:
            batch_images = batch_images.reshape(-1, num_features)
            
            # 前向传播
            y_pred = model(batch_images)
            
            # 计算损失和准确率
            loss = cross_entropy(y_pred, batch_labels)
            acc = accuracy(y_pred, batch_labels)
            
            total_loss += loss.item()
            total_accuracy += acc.item()
    
    avg_loss = total_loss / len(test_dataloader)
    avg_accuracy = total_accuracy / len(test_dataloader)
    
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {avg_accuracy:.4f}")




In [17]:
# 下载训练集和测试集
train_dataset = FashionMNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = FashionMNIST(root='data/', train=False, transform=ToTensor(), download=True)

# 创建训练集和测试集的数据加载器
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# 初始化模型参数
num_features = 28 * 28  # 输入特征的数量（图像大小为28x28）
num_classes = 10  # 类别数量
w = torch.randn(num_features, num_classes, requires_grad=True)
b = torch.zeros(num_classes, requires_grad=True)
# 设置训练参数并开始训练
epochs = 50
learning_rate = 0.1

train(epochs, learning_rate)

# 进行测试
test()

Epoch [1/50], Loss: 2.2780, Accuracy: 0.6116
Epoch [2/50], Loss: 1.2286, Accuracy: 0.7329
Epoch [3/50], Loss: 1.0433, Accuracy: 0.7605
Epoch [4/50], Loss: 0.9429, Accuracy: 0.7751
Epoch [5/50], Loss: 0.8732, Accuracy: 0.7845
Epoch [6/50], Loss: 0.8232, Accuracy: 0.7929
Epoch [7/50], Loss: 0.7823, Accuracy: 0.7985
Epoch [8/50], Loss: 0.7500, Accuracy: 0.8034
Epoch [9/50], Loss: 0.7217, Accuracy: 0.8070
Epoch [10/50], Loss: 0.6973, Accuracy: 0.8099
Epoch [11/50], Loss: 0.6771, Accuracy: 0.8131
Epoch [12/50], Loss: 0.6583, Accuracy: 0.8156
Epoch [13/50], Loss: 0.6418, Accuracy: 0.8180
Epoch [14/50], Loss: 0.6272, Accuracy: 0.8205
Epoch [15/50], Loss: 0.6145, Accuracy: 0.8225
Epoch [16/50], Loss: 0.6025, Accuracy: 0.8230
Epoch [17/50], Loss: 0.5914, Accuracy: 0.8259
Epoch [18/50], Loss: 0.5819, Accuracy: 0.8256
Epoch [19/50], Loss: 0.5719, Accuracy: 0.8274
Epoch [20/50], Loss: 0.5640, Accuracy: 0.8296
Epoch [21/50], Loss: 0.5568, Accuracy: 0.8306
Epoch [22/50], Loss: 0.5485, Accuracy: 0.83

In [18]:
# 保存模型
torch.save({'w': w, 'b': b}, 'Classification_model.pth')

In [20]:
# 加载模型
checkpoint = torch.load('Classification_model.pth')

# 下载训练集和测试集
train_dataset = FashionMNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = FashionMNIST(root='data/', train=False, transform=ToTensor(), download=True)

# 创建训练集和测试集的数据加载器
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# 初始化模型参数
num_features = 28 * 28  # 输入特征的数量（图像大小为28x28）
num_classes = 10  # 类别数量
w = checkpoint['w']
b = checkpoint['b']
# 设置训练参数并开始训练
epochs = 50
learning_rate = 0.1

train(epochs, learning_rate)

# 进行测试
test()

Epoch [1/50], Loss: 0.4445, Accuracy: 0.8525
Epoch [2/50], Loss: 0.4437, Accuracy: 0.8516
Epoch [3/50], Loss: 0.4430, Accuracy: 0.8517
Epoch [4/50], Loss: 0.4404, Accuracy: 0.8531
Epoch [5/50], Loss: 0.4378, Accuracy: 0.8537
Epoch [6/50], Loss: 0.4393, Accuracy: 0.8530
Epoch [7/50], Loss: 0.4368, Accuracy: 0.8535
Epoch [8/50], Loss: 0.4351, Accuracy: 0.8549
Epoch [9/50], Loss: 0.4330, Accuracy: 0.8549
Epoch [10/50], Loss: 0.4315, Accuracy: 0.8553
Epoch [11/50], Loss: 0.4301, Accuracy: 0.8559
Epoch [12/50], Loss: 0.4295, Accuracy: 0.8555
Epoch [13/50], Loss: 0.4274, Accuracy: 0.8559
Epoch [14/50], Loss: 0.4276, Accuracy: 0.8547
Epoch [15/50], Loss: 0.4259, Accuracy: 0.8566
Epoch [16/50], Loss: 0.4234, Accuracy: 0.8568
Epoch [17/50], Loss: 0.4241, Accuracy: 0.8559
Epoch [18/50], Loss: 0.4227, Accuracy: 0.8573
Epoch [19/50], Loss: 0.4227, Accuracy: 0.8572
Epoch [20/50], Loss: 0.4209, Accuracy: 0.8564
Epoch [21/50], Loss: 0.4193, Accuracy: 0.8573
Epoch [22/50], Loss: 0.4187, Accuracy: 0.85

In [21]:
# 保存模型
torch.save({'w': w, 'b': b}, 'Classification_model.pth')

In [8]:
# 加载模型
checkpoint = torch.load('Classification_model.pth')

# 下载训练集和测试集
train_dataset = FashionMNIST(root='data/', train=True, transform=ToTensor(), download=True)
test_dataset = FashionMNIST(root='data/', train=False, transform=ToTensor(), download=True)

# 创建训练集和测试集的数据加载器
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# 初始化模型参数
num_features = 28 * 28  # 输入特征的数量（图像大小为28x28）
num_classes = 10  # 类别数量
w = checkpoint['w']
b = checkpoint['b']
# 设置训练参数并开始训练
epochs = 50
learning_rate = 0.1

train(epochs, learning_rate)

# 进行测试
test()

KeyboardInterrupt: 

In [11]:
w.size()

torch.Size([784, 10])

In [12]:
w

tensor([[-0.7056,  3.6875,  1.1252,  ..., -0.0564, -0.7181, -0.7160],
        [-1.0933, -0.2097, -0.3350,  ...,  0.5351,  1.9643, -0.1643],
        [ 1.0277, -1.7061, -1.8516,  ...,  0.5984, -0.3432, -0.4054],
        ...,
        [-0.2210,  1.4967,  0.6180,  ..., -1.0178, -0.8755,  0.7527],
        [-0.8656,  0.2825, -0.6440,  ..., -0.5004, -0.3248,  0.5127],
        [-0.4026, -1.1222,  1.7200,  ...,  0.3806, -1.2070,  1.0543]],
       requires_grad=True)